In [ ]:
#Keep relevant variables
df = df[['TEAM_ID', 'TEAM_ABBREVIATION', 'TEAM_NAME', 'GAME_ID',
       'GAME_DATE', 'MATCHUP', 'WL', 'MIN', 'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]

In [1]:
from nba_api.stats.endpoints import leaguegamelog
import pandas as pd
import numpy as np

In [2]:
games = leaguegamelog.LeagueGameLog(season = '2018')

In [3]:
df = pd.DataFrame(games.get_data_frames()[0])

In [7]:
teams = df['TEAM_NAME'].unique()

In [ ]:
teams

In [ ]:
merged = home_df.merge(away_df, on = "GAME_ID",suffixes = ("_H", "_A"))

In [ ]:
merged = merged.sort_values(by = ['GAME_DATE'])

In [4]:
#Can use this to get means and standard deviations
def generate_game_stats(game_df, teams): 
    #all_stats_arr = np.empty((30, 77, 19))
    all_teams_stats_df = pd.DataFrame()
    for i, team in enumerate(teams):
        temp_df = df[df['TEAM_NAME'] == team]
        temp_df = temp_df.sort_values(by = ['GAME_DATE'])
        temp_df = temp_df.set_index('GAME_ID')
        key_data = temp_df[['GAME_DATE', 'TEAM_NAME', 'MATCHUP', 'WL']].iloc[5:]
        temp_df = temp_df[[
           'FGM', 'FGA', 'FG_PCT', 'FG3M',
           'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
           'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
        #final_stats_df = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]#.to_dict('list')
        #single_stats_arr = final_stats_df.to_numpy()
        #all_stats_arr[i] = single_stats_arr
        
        final_organized_stats = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]#.to_dict('index')
        stats_with_key = pd.concat([final_organized_stats, key_data], axis = 1)
     
        all_teams_stats_df = pd.concat([all_teams_stats_df, stats_with_key], axis = 0)
        
    home_df = all_teams_stats_df[all_teams_stats_df['MATCHUP'].str.contains("vs.")]
    away_df = all_teams_stats_df[all_teams_stats_df['MATCHUP'].str.contains("@")]
    merged_game_stats_df = home_df.merge(away_df, on = "GAME_ID",suffixes = ("_H", "_A"))
    merged_game_stats_df = merged_game_stats_df.sort_values(by = ['GAME_DATE_H'])
      
    return merged_game_stats_df

In [5]:
#Function to get population mean and standard deviations - might be stupid but I think it makes sense
#Make sure to use this on train and test set separately 
#After this, make these stats for each game their own matrices, so I could do (game_matrix - mean_matrix / std_matrix)
def generate_population_statistics(stats_df):
    game_means = np.empty((stats_df.shape[0], 19))
    game_stds = np.empty((stats_df.shape[0], 19))
    for i in range(stats_df.shape[0]):
        team_dict = {}
        pop_stats_arr = np.empty((30, 19))
        it1 = i
        it2 = i+1
        team_it = 0
             
        while len(team_dict.keys()) < 30:
            if it1 >= 0:
                if stats_df.iloc[it1]['TEAM_NAME_H'] not in team_dict.keys():  
                    team_dict[stats_df.iloc[it1]['TEAM_NAME_H']] = ''
                    
                    pop_stats_arr[team_it] = stats_df.iloc[it1][['FGM_H', 'FGA_H', 'FG_PCT_H', 'FG3M_H', 'FG3A_H', 'FG3_PCT_H', 'FTM_H',
                   'FTA_H', 'FT_PCT_H', 'OREB_H', 'DREB_H', 'REB_H', 'AST_H', 'STL_H',
                   'BLK_H', 'TOV_H', 'PF_H', 'PTS_H', 'PLUS_MINUS_H']].to_numpy()
                    team_it += 1
                    
                if stats_df.iloc[it1]['TEAM_NAME_A'] not in team_dict.keys():
                    team_dict[stats_df.iloc[it1]['TEAM_NAME_A']] = ''
                    
                    pop_stats_arr[team_it] = stats_df.iloc[it1][['FGM_A', 'FGA_A', 'FG_PCT_A',
                   'FG3M_A', 'FG3A_A', 'FG3_PCT_A', 'FTM_A', 'FTA_A', 'FT_PCT_A', 'OREB_A',
                   'DREB_A', 'REB_A', 'AST_A', 'STL_A', 'BLK_A', 'TOV_A', 'PF_A', 'PTS_A',
                   'PLUS_MINUS_A']].to_numpy()
                    team_it += 1
                    
                it1 -= 1    
                           
            if it2 < stats_df.shape[0]:
                if stats_df.iloc[it2]['TEAM_NAME_H'] not in team_dict.keys():  
                    team_dict[stats_df.iloc[it2]['TEAM_NAME_H']] = ''
                    
                    pop_stats_arr[team_it] = stats_df.iloc[it2][['FGM_H', 'FGA_H', 'FG_PCT_H', 'FG3M_H', 'FG3A_H', 'FG3_PCT_H', 'FTM_H',
                   'FTA_H', 'FT_PCT_H', 'OREB_H', 'DREB_H', 'REB_H', 'AST_H', 'STL_H',
                   'BLK_H', 'TOV_H', 'PF_H', 'PTS_H', 'PLUS_MINUS_H']].to_numpy()
                    team_it += 1
                    
                if stats_df.iloc[it2]['TEAM_NAME_A'] not in team_dict.keys():
                    team_dict[stats_df.iloc[it2]['TEAM_NAME_A']] = ''
                    
                    pop_stats_arr[team_it] = stats_df.iloc[it2][['FGM_A', 'FGA_A', 'FG_PCT_A',
                   'FG3M_A', 'FG3A_A', 'FG3_PCT_A', 'FTM_A', 'FTA_A', 'FT_PCT_A', 'OREB_A',
                   'DREB_A', 'REB_A', 'AST_A', 'STL_A', 'BLK_A', 'TOV_A', 'PF_A', 'PTS_A',
                   'PLUS_MINUS_A']].to_numpy()
                    team_it += 1
                    
                it2 += 1
                
        mean_stats = np.mean(pop_stats_arr, axis = 0)        
        std_stats = np.std(pop_stats_arr, axis = 0)
        
        game_means[i] = mean_stats
        game_stds[i] = std_stats
        
    return game_means, game_stds    
    

In [117]:
#Function to get z scores and finalize attributes as (Home minus Away)
def normalize_and_standardize(stats_df, game_means, game_stds):
   
    home_stats = np.empty((stats_df.shape[0], 19))
    away_stats = np.empty((stats_df.shape[0], 19))
    
    home_stats = stats_df.iloc[:, 0:19]
    away_stats = stats_df.iloc[:, 23:42]
    win_loss_home = stats_df.iloc[:, 22]
    matchup = stats_df.iloc[:, [43, 20]]
    
    #standardize home and away stats 
    home_z = np.divide((home_stats - game_means), game_stds)
    
    away_z = np.divide((away_stats - game_means), game_stds)
    
    final = np.subtract(home_z, away_z)
    
    final = pd.DataFrame(final)

    final = pd.concat([final, win_loss_home], axis = 1)
    
    return final, matchup

In [8]:
stats_df = generate_game_stats(df, teams)

In [112]:
stats_df.iloc[:, [20,43]]

,TEAM_NAME_H,TEAM_NAME_A
GAME_ID,,
0021800068,New York Knicks,Golden State Warriors
0021800078,Philadelphia 76ers,Charlotte Hornets
0021800077,Cleveland Cavaliers,Indiana Pacers
0021800081,Milwaukee Bucks,Orlando Magic
0021800084,Dallas Mavericks,Utah Jazz
...,...,...
0021801225,Memphis Grizzlies,Golden State Warriors
0021801228,Denver Nuggets,Minnesota Timberwolves
0021801224,Philadelphia 76ers,Chicago Bulls


In [18]:
stats_df.columns

Index(['FGM_H', 'FGA_H', 'FG_PCT_H', 'FG3M_H', 'FG3A_H', 'FG3_PCT_H', 'FTM_H',
       'FTA_H', 'FT_PCT_H', 'OREB_H', 'DREB_H', 'REB_H', 'AST_H', 'STL_H',
       'BLK_H', 'TOV_H', 'PF_H', 'PTS_H', 'PLUS_MINUS_H', 'GAME_DATE_H',
       'TEAM_NAME_H', 'MATCHUP_H', 'WL_H', 'FGM_A', 'FGA_A', 'FG_PCT_A',
       'FG3M_A', 'FG3A_A', 'FG3_PCT_A', 'FTM_A', 'FTA_A', 'FT_PCT_A', 'OREB_A',
       'DREB_A', 'REB_A', 'AST_A', 'STL_A', 'BLK_A', 'TOV_A', 'PF_A', 'PTS_A',
       'PLUS_MINUS_A', 'GAME_DATE_A', 'TEAM_NAME_A', 'MATCHUP_A', 'WL_A'],
      dtype='object')

In [11]:
game_means, game_stds = generate_population_statistics(stats_df)

In [12]:
game_means

array([[ 4.11000000e+01,  9.03400000e+01,  4.55646667e-01, ...,
         2.26533333e+01,  1.12140000e+02, -8.00000000e-02],
       [ 4.11000000e+01,  9.03400000e+01,  4.55646667e-01, ...,
         2.26533333e+01,  1.12140000e+02, -8.00000000e-02],
       [ 4.11000000e+01,  9.03400000e+01,  4.55646667e-01, ...,
         2.26533333e+01,  1.12140000e+02, -8.00000000e-02],
       ...,
       [ 4.23333333e+01,  9.05266667e+01,  4.68820000e-01, ...,
         1.94400000e+01,  1.13126667e+02,  2.53333333e-01],
       [ 4.23333333e+01,  9.05266667e+01,  4.68820000e-01, ...,
         1.94400000e+01,  1.13126667e+02,  2.53333333e-01],
       [ 4.23333333e+01,  9.05266667e+01,  4.68820000e-01, ...,
         1.94400000e+01,  1.13126667e+02,  2.53333333e-01]])

In [118]:
final, matchup = normalize_and_standardize(stats_df, game_means, game_stds)

C:\Users\batar\AppData\Local\Temp\ipykernel_8300\1595565284.py:17: FutureWarning: Calling a ufunc on non-aligned DataFrames (or DataFrame/Series combination). Currently, the indices are ignored and the result takes the index/columns of the first DataFrame. In the future , the DataFrames/Series will be aligned before applying the ufunc.
Convert one of the arguments to a NumPy array (eg 'ufunc(df1, np.asarray(df2)') to keep the current behaviour, or align manually (eg 'df1, df2 = df1.align(df2)') before passing to the ufunc to obtain the future behaviour and silence this warning.
  final = np.subtract(home_z, away_z)


In [119]:
matchup

,TEAM_NAME_A,TEAM_NAME_H
GAME_ID,,
0021800068,Golden State Warriors,New York Knicks
0021800078,Charlotte Hornets,Philadelphia 76ers
0021800077,Indiana Pacers,Cleveland Cavaliers
0021800081,Orlando Magic,Milwaukee Bucks
0021800084,Utah Jazz,Dallas Mavericks
...,...,...
0021801225,Golden State Warriors,Memphis Grizzlies
0021801228,Minnesota Timberwolves,Denver Nuggets
0021801224,Chicago Bulls,Philadelphia 76ers


In [ ]:
stats_arr

In [ ]:
#Function to flatten stats dict and find averages. Would show average and std dev. of team atttributes for 5th, 
# 6th... 82nd game

num_teams = teams.size

In [ ]:
#test
team = 'Oklahoma City Thunder'
temp_df = df[df['TEAM_NAME'] == team]
temp_df = temp_df.sort_values(by = ['GAME_DATE'])
temp_df = temp_df.set_index('GAME_ID')
temp_df = temp_df[[
           'FGM', 'FGA', 'FG_PCT', 'FG3M',
           'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
           'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
final_stats_df = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]
single_stats_arr = final_stats_df.to_numpy()
all_stats_arr = np.empty((30, 77, 19))


In [ ]:
stats_calc_dict = {}
descriptive_dict = {}
all_stats_arr = np.empty((30, 77, 19))
team = 'Oklahoma City Thunder'
temp_df = df[df['TEAM_NAME'] == team]
temp_df = temp_df.sort_values(by = ['GAME_DATE'])
temp_df = temp_df.set_index('GAME_ID')
key_data = temp_df[['TEAM_NAME', 'MATCHUP', 'WL']].iloc[5:]
temp_df = temp_df[[
           'FGM', 'FGA', 'FG_PCT', 'FG3M',
           'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
           'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
final_stats_df = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]
single_stats_arr = final_stats_df.to_numpy()
all_stats_arr[0] = single_stats_arr
        
final_organized_stats = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]
stats_with_key = pd.concat([final_organized_stats, key_data], axis = 1)

In [ ]:
all_teams_stats_df = pd.DataFrame()

In [ ]:
all_teams_stats_df = pd.concat([all_teams_stats_df, stats_with_key], axis = 0)

In [ ]:
temp_df = df[df['TEAM_NAME'] == team]
temp_df.head(10)

In [ ]:
stats_with_key

In [ ]:
final_organized_stats = temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]

In [ ]:
final_organized_stats

In [ ]:
final_stats_df.to_numpy().shape

In [ ]:
df.from_dict(dict2['Oklahoma City Thunder'], orient = 'index')

In [ ]:
dict1, dict2 = generate_team_stats(df, teams)

In [ ]:
dict2

In [ ]:
temp_df = df[df['TEAM_NAME'] == 'Oklahoma City Thunder']

In [ ]:
temp_df = temp_df.set_index('GAME_ID')

In [ ]:
game_ids = temp_df['GAME_ID']

In [ ]:
temp_df = temp_df[[
           'FGM', 'FGA', 'FG_PCT', 'FG3M',
           'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
           'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]
temp_df.rolling(5).mean().shift(periods = 1).iloc[5:]

In [ ]:
game_ids.shift(periods = 5).iloc[5:]

In [ ]:
temp_df = temp_df.sort_values(by = ['GAME_DATE'])

In [ ]:
temp_df = temp_df[[
       'FGM', 'FGA', 'FG_PCT', 'FG3M',
       'FG3A', 'FG3_PCT', 'FTM', 'FTA', 'FT_PCT', 'OREB', 'DREB', 'REB', 'AST',
       'STL', 'BLK', 'TOV', 'PF', 'PTS', 'PLUS_MINUS']]

In [ ]:
temp_df.rolling(5).mean().shift(periods = 1).tail(10)

In [ ]:
df.head(5)

In [ ]:
#Split data frames into home and away teams
home_df = df[df['MATCHUP'].str.contains("vs.")]
away_df = df[df['MATCHUP'].str.contains("@")]

In [ ]:
merged = home_df.merge(away_df, on = "GAME_ID",suffixes = ("_H", "_A"))

In [ ]:
merged

In [ ]:
#make list of all team names, for loop filter by team n times, and make the rolling dict values?
# Can then go in and replace values/make new df with the normalized vals